In [ ]:
import os 
import sys
import cv2
import numpy as np
import neuroglancer
from skimage import io, measure
import matplotlib.pyplot as plt

HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)


from utilities.SqlController import SqlController

In [ ]:
def mask_to_shell(mask):
    sub_contours = measure.find_contours(mask, 1)

    sub_shells = []
    for sub_contour in sub_contours:
        sub_contour.T[[0, 1]] = sub_contour.T[[1, 0]]
        pts = sub_contour.astype(np.int32).reshape((-1, 1, 2))

        sub_shell = np.zeros(mask.shape, dtype='uint8')
        sub_shell = cv2.polylines(sub_shell, [pts], True, 1, 5, lineType=cv2.LINE_AA)
        sub_shells.append(sub_shell)
    shell = np.array(sub_shells).sum(axis=0)
    
    return shell

In [ ]:
VOL_DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK39/preps/rotated_aligned_masked'
files = sorted(os.listdir(VOL_DIR))

volume = []
for file in files:
    tif = cv2.imread(os.path.join(VOL_DIR, file), cv2.IMREAD_GRAYSCALE)
    tif = mask_to_shell(tif)
    volume.append(tif)
volume = np.array(volume).astype('uint8')
volume = np.swapaxes(volume, 0, 2)

In [ ]:
animal = 'DK39'
downsample_factor = 32
precompute_path = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK39/neuroglancer_data/shell'

sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000

ng = NumpyToNeuroglancer(volume, scales)
# ng.preview()
ng.init_precomputed(precompute_path)
# ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()